In [1]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 

In [5]:
results_dir = "../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities_gemma.csv")
sim_df = pd.read_csv(f"{results_dir}/context_response_similarity_scores.csv")

cities_df = pd.read_csv("../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

sar_results_dir = "../../european-city-data/rag-sustainability/results/results-combined_prompts_SAR/"
sar_results = pd.read_csv(f"{sar_results_dir}/recommended_cities_sar_gemma.csv")

sim_sar = pd.read_csv(f"{sar_results_dir}/context_response_similarity_scores.csv")

In [6]:
combined_df = pd.merge(
    left=results_df, 
    right=sar_results,
    how='left',
    left_on=['model', 'prompt_id'],
    right_on=['model', 'prompt_id'],
)
combined_df.head()

,model,prompt_id,rec_cities,response,rec_cities_sar,response_sar
0,gemma2,prompt_17_gemini-ui,"['Gaziantep', 'Konya']",It seems you've provided information about two...,"['Gaziantep', 'Konya']",The text you provided gives information about ...
1,gemma2,prompt_27_gemini-1.5-pro-001,"['Coimbra', 'Varna']",It looks like you've provided information abou...,"['Coimbra', 'Varna']",Let's break down how to structure this informa...
2,gemma2,prompt_42_gpt-4o-mini,['Amsterdam'],The provided text gives a list of events happe...,['Amsterdam'],The provided text outlines a list of events ha...
3,gemma2,prompt_0_gpt-4o-mini,['Strasbourg'],It seems like you're providing text snippets a...,['Strasbourg'],Let's break down how to structure this informa...
4,gemma2,prompt_44_gemini-1.5-pro-001,['Prague'],It looks like you've provided information abou...,['Prague'],This data looks like a collection of tourist a...


In [7]:
results_df.head()

,model,prompt_id,rec_cities,response
0,gemma2,prompt_17_gemini-ui,"['Gaziantep', 'Konya']",It seems you've provided information about two...
1,gemma2,prompt_27_gemini-1.5-pro-001,"['Coimbra', 'Varna']",It looks like you've provided information abou...
2,gemma2,prompt_42_gpt-4o-mini,['Amsterdam'],The provided text gives a list of events happe...
3,gemma2,prompt_0_gpt-4o-mini,['Strasbourg'],It seems like you're providing text snippets a...
4,gemma2,prompt_44_gemini-1.5-pro-001,['Prague'],It looks like you've provided information abou...


In [13]:
# Compute the average number of cities from our database that have been recommended by each model 

llama = combined_df[combined_df['model'] == 'llama3point1-instruct']
mistral = combined_df[combined_df['model'] == 'mistral-instruct']
gemma = combined_df[combined_df['model'] == 'gemma2']
llama

,model,prompt_id,rec_cities,response,rec_cities_sar,response_sar
200,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,"['Kaunas', 'Sibiu']",I recommend Kaunas and Sibiu. I recommend the...
201,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...","['Kahramanmaras', 'Nevsehir']","I recommend Baia Mare, Kahramanmaras, and Nev..."
202,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Arkhangelsk', 'Amsterdam']","I recommend Nalchik, Arkhangelsk, and Amsterd..."
203,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Sibiu']","I recommend Kaunas, Oradea, and Sibiu. I reco..."
204,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,"['Kaunas', 'Rennes', 'Copenhagen']","I recommend Kaunas, Rennes, and Copenhagen fo..."
...,...,...,...,...,...,...
395,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...","['Innsbruck', 'Strasbourg']",I recommend Innsbruck and Strasbourg because ...
396,llama3point1-instruct,prompt_44_gemini-ui,['Stavanger'],"I recommend Stavanger, Norway, because it off...","['Nalchik', 'Kaunas', 'Stavanger', 'Erzurum', ...","I recommend Nalchik, Kaunas, and Stavanger. I..."
397,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Munich', 'Thessaloniki', 'Arkhangelsk', 'Gaz...","I recommend Munich, Germany for its vibrant a...","['Oradea', 'Gaziantep', 'Varna']","I recommend Oradea, Gaziantep, and Varna. I r..."
398,llama3point1-instruct,prompt_50_gemini-ui,"['Varna', 'Tallinn', 'London', 'Cork', 'Innsbr...","I recommend Varna, Bulgaria for your weekend ...","['Tallinn', 'Cork']","I recommend Tallinn, Baia Mare, and Cork. I r..."


### How many cities does each model recommend on average? 

For reference: the prompt asked each model to recommend 3 cities to the user 

In [14]:
# Average number of cities in context per prompt - Llama

llama_avg_cities = np.mean(llama['rec_cities'].apply(lambda x: len(ast.literal_eval(x))))
print(llama_avg_cities)

llama_avg_cities_sus = np.mean(llama['rec_cities_sar'].apply(lambda x: len(ast.literal_eval(x))))
print(llama_avg_cities_sus)



2.83
2.93


In [15]:
# Average number of cities in context per prompt - Mistral

mistral_avg_cities = np.mean(mistral['rec_cities'].apply(lambda x: len(ast.literal_eval(x))))
print(mistral_avg_cities)

mistral_avg_cities_sus = np.mean(mistral['rec_cities_sar'].apply(lambda x: len(ast.literal_eval(x))))
print(mistral_avg_cities_sus)

2.58
3.49


In [16]:
# Average number of cities in context per prompt - Gemma

gemma_avg_cities = np.mean(gemma['rec_cities'].apply(lambda x: len(ast.literal_eval(x))))
print(gemma_avg_cities)

gemma_avg_cities_sus = np.mean(gemma['rec_cities_sar'].apply(lambda x: len(ast.literal_eval(x))))
print(gemma_avg_cities_sus)

1.305
1.245


Llama 3.1. shows a slightly better performance as opposed to Mistral but both are below the expected requirement of 3 recommended cities per prompt

### How many out-of-context cities are recommended to the user? 

How to compute this without ambiguities???

In [17]:
def avg_city_not_in_db(rec_cities, ref_cities = eu_cities):
    size = len(ast.literal_eval(rec_cities))
    if size == 0: 
        # return 0
        return []
    
    cities_in_db = [city for city in ast.literal_eval(rec_cities) if city not in ref_cities]
    # cities_in_db = sum(1 for city in ast.literal_eval(rec_cities) if city not in ref_cities)
    return cities_in_db



# # avg_llama = llama['rec_cities'].apply(avg_city_not_in_db)
# # avg_llama_sus = llama['rec_cities_sustainable'].apply(avg_city_not_in_db)

# avg_mistral = mistral['rec_cities'].apply(avg_city_not_in_db)
# avg_mistral_sus = mistral['rec_cities_sustainable'].apply(avg_city_not_in_db)

# # avg_table = np.array([avg_llama, avg_llama_sus, avg_mistral, avg_mistral_sus]).reshape(2, 2)
# # print(avg_table)

# print(avg_mistral)



In [18]:
def flatten_list(cities_list):
     return set([city for cities in cities_list for city in ast.literal_eval(cities)])

def cities_not_in_db(cities_list):
    total_rec_cities = flatten_list(cities_list)
    cities_not_in_db = total_rec_cities - set(eu_cities)
    return cities_not_in_db

# llama_db = cities_not_in_db(list(llama['rec_cities']))
# llama_db_sus = cities_not_in_db(list(llama['rec_cities_sustainable']))

# mistral_db = cities_not_in_db(list(mistral['rec_cities']))
# mistral_db_sus = cities_not_in_db(list(mistral['rec_cities_sustainable']))

### Count of empty lists for cities => none of the retrieved cities from the context were recommended => model has hallucinated

In [21]:
def hallucination(df):
    prompt_ids = []

    for i, row in df.iterrows():
        if len(ast.literal_eval(row['rec_cities_sar'])) == 0:
            prompt_ids.append(row['prompt_id'])
    
    return prompt_ids

In [22]:
llama_hal_prompts = hallucination(llama)
llama_hal_prompts

[]

In [33]:
print(hallucination(gemma))
len(hallucination(gemma))

['prompt_49_gemini-1.5-pro-001', 'prompt_36_gemini-ui', 'prompt_55_gemini-ui', 'prompt_27_gpt-4o-mini', 'prompt_47_gpt-4o-mini', 'prompt_5_gpt-4o-mini', 'prompt_6_gemini-ui', 'prompt_21_gpt-4o-mini', 'prompt_66_gpt-4o-mini', 'prompt_28_gemini-ui', 'prompt_41_gpt-4o-mini', 'prompt_52_gemini-1.5-pro-001', 'prompt_61_gpt-4o-mini', 'prompt_8_gemini-1.5-pro-001', 'prompt_32_gemini-ui', 'prompt_71_gpt-4o-mini', 'prompt_51_gemini-ui', 'prompt_33_gpt-4o-mini', 'prompt_4_gemini-ui', 'prompt_56_gemini-ui', 'prompt_33_gemini-ui', 'prompt_35_gpt-4o-mini', 'prompt_77_gpt-4o-mini']


23

In [25]:
count_empty = lambda x: 1 if len(ast.literal_eval(x)) == 0 else 0

mistral_empty = sum(mistral['rec_cities'].apply(count_empty))
mistral_empty_sus = sum(mistral['rec_cities_sar'].apply(count_empty))

llama_empty = sum(llama['rec_cities'].apply(count_empty))
llama_empty_sus = sum(llama['rec_cities_sar'].apply(count_empty))

gemma_empty = sum(gemma['rec_cities'].apply(count_empty))
gemma_empty_sus = sum(gemma['rec_cities_sar'].apply(count_empty))

In [29]:
table_empty_count = np.array([llama_empty, llama_empty_sus, mistral_empty, mistral_empty_sus, gemma_empty, gemma_empty_sus]).reshape(3, 2)
table_empty_count

array([[ 0,  0],
       [28, 19],
       [23, 23]])

Mistral is worse than Llama in this regard and hallucinates in 28/200 prompts without sustainability and 18/200 prompts with sustainability, where as ALL of the prompt responses in Llama include atleast 1 city from the retrieved context.

While hallucination comes down in Mistral when sustainability is added, it seems to increase by 100% in Llama - what could be a possible reason??

## With SAR

In [30]:
llama = combined_df[combined_df['model'] == 'llama3point1-instruct']
mistral = combined_df[combined_df['model'] == 'mistral-instruct']
gemma = combined_df[combined_df['model'] == 'gemma2']

In [34]:
count_empty = lambda x: 1 if len(ast.literal_eval(x)) == 0 else 0

mistral_empty = sum(mistral['rec_cities'].apply(count_empty))
mistral_empty_sar = sum(mistral['rec_cities_sar'].apply(count_empty))

llama_empty = sum(llama['rec_cities'].apply(count_empty))
llama_empty_sar = sum(llama['rec_cities_sar'].apply(count_empty))

gemma_empty = sum(gemma['rec_cities'].apply(count_empty))
gemma_empty_sus = sum(gemma['rec_cities_sar'].apply(count_empty))

In [35]:
table_empty_count = np.array([llama_empty, llama_empty_sus, mistral_empty, mistral_empty_sus, gemma_empty, gemma_empty_sus]).reshape(3, 2)
table_empty_count

array([[ 0,  0],
       [28, 19],
       [23, 23]])